In [ ]:
# 🚀 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

# 🚀 2. 데이터 로드 및 전처리
def load_and_preprocess(csv_path: str):
    df = pd.read_csv(csv_path)

    # 예측에 쓸 변수만 선택
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
    X = df[features].copy()

    # 범주형 변수 원-핫 인코딩
    X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)

    # Age 결측치는 평균으로 채우기
    if X['Age'].isna().any():
        X['Age'].fillna(X['Age'].mean(), inplace=True)

    y = df['Survived'].astype(int).values

    # 입력 데이터 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled.astype(np.float32), y.astype(np.float32)

# 🚀 3. 딥러닝 모델 정의
def build_model(input_dim: int) -> tf.keras.Model:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # 이진 분류
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# 🚀 4. 파일 경로 지정
csv_path = 'titanic1309.csv'  # 자신의 경로로 수정!

# 🚀 5. 데이터 준비
X, y = load_and_preprocess(csv_path)

# 🚀 6. 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 🚀 7. 모델 생성 및 학습
model = build_model(X_train.shape[1])
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True
        )
    ],
    verbose=1
)

# 🚀 8. 테스트 데이터로 평가
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {acc:.4f}\n")

# 🚀 9. 분류 보고서 및 혼동 행렬
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 🚀 10. 샘플 예측 결과 (5개 랜덤)
sample_idx = np.random.choice(len(X_test), size=5, replace=False)
print("\nSample Predictions (Survived=1, Died=0):")
for i in sample_idx:
    print(f"True: {int(y_test[i])}, Predicted: {int(y_pred[i])}")
